Importamos librerías

In [2]:
import awswrangler as wr
import boto3
import pandas as pd

Cargamos nuestras credenciales de AWS

In [3]:
my_session = boto3.Session(
    aws_access_key_id = "access_key",
    aws_secret_access_key = "secret_key",
    region_name = "region_name"
)

In [8]:
s3_client = boto3.client(
    's3',
    aws_access_key_id= "access_key",
    aws_secret_access_key= "secret_key"
)

Creamos bucket donde irán los datasets

In [ ]:
s3_client.create_bucket(Bucket="Bucket Name")

Verificamos que estén cargados

In [ ]:
wr.s3.list_objects("s3://Bucket Name", boto3_session=my_session)

#### Aplicamos el proceso de **ETL** a cada dataset de nuestro bucket de S3

**Tabla Gini**

In [ ]:
#Cargamos nuestro dataset
df_gini = wr.s3.read_csv(path="s3://data-fm-05/raw_data/gini.csv", boto3_session=my_session)

#Hacemos transformaciones
df_gini.rename(columns={'Entity':'pais','Code':'codigo_pais', 'Year':'anio', 'Gini coefficient': 'valor'}, inplace=True)
df_gini.fillna(0, inplace= True)

#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=df_gini,path="s3://data-fm-05/clean_data/gini.csv", boto3_session=my_session)

**Tabla PIB**

In [ ]:
#Cargamos nuestro dataset
df_PIB = wr.s3.read_csv(path="s3://data-fm-05/raw_data/Pib.csv", boto3_session=my_session)

#Hacemos transformaciones
df_PIB.drop(['Indicator Name', 'Indicator Code', 'Unnamed: 66'], axis=1, inplace= True)
df_PIB.drop(columns=df_PIB.columns[2:32],axis=1,inplace= True)
df_PIB.drop([1,3], axis=0, inplace=True)
df_PIB.fillna(0, inplace= True)
df_PIB.rename(columns={'Country Name':'pais','Country Code':'codigo_pais'}, inplace=True)

#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=df_PIB, path="s3://data-fm-05/clean_data/Pib.csv", boto3_session=my_session)

**Tabla Empleo por sector**

In [ ]:
#Cargamos nuestro dataset
df_empleo_sector = wr.s3.read_csv(path='s3://data-fm-05/raw_data/Employment_for_sector.csv', sep= ';', boto3_session=my_session)

#Hacemos transformaciones
df_empleo_sector.drop(['Series Code', '1990 [YR1990]'], axis=1, inplace= True)
df_empleo_sector.drop(df_empleo_sector.index[1596:1601], axis=0, inplace=True)
df_empleo_sector.rename(columns={'Series Name':'sector',
                              'Country Name':'pais', 
                              'Country Code':'codigo_pais',
                              '1990 [YR1990]':'1990', '2000 [YR2000]':'2000', '2012 [YR2012]':'2012',
                              '2013 [YR2013]':'2013', '2014 [YR2014]':'2014', '2015 [YR2015]':'2015',
                              '2016 [YR2016]':'2016', '2017 [YR2017]':'2017', '2018 [YR2018]':'2018',
                              '2019 [YR2019]':'2019', '2020 [YR2020]':'2020', '2021 [YR2021]':'2021'}, inplace=True)
df_empleo_sector.replace({"..": 0}, inplace= True)

#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=df_empleo_sector, path='s3://data-fm-05/clean_data/Employment_for_sector.csv', boto3_session=my_session)

**Tabla Industria migracion**

In [ ]:
#Cargamos nuestro dataset
df_industria = wr.s3.read_csv(path='s3://data-fm-05/raw_data/industry_migration_public.csv', boto3_session=my_session)

#Hacemos transformaciones
df_industria.drop(['wb_income', 'wb_region', 'isic_section_index','industry_id'], axis=1, inplace=True )
df_industria.drop(columns=df_industria.columns[9:26],axis=1,inplace= True)
df_industria.rename(columns={'country_code ':'codigo_pais','country_name ':'pais',
                          'isic_section_name':'sector', 'industry_name':'industria',
                          'net_per_10K_2015': '2015', 'net_per_10K_2016': '2016', 'net_per_10K_2017': '2017',
                          'net_per_10K_2018': '2018', 'net_per_10K_2019': '2019'}, inplace=True)

#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=df_industria, path='s3://data-fm-05/clean_data/industry_migration_public.csv', boto3_session=my_session)

**Tabla de habilidades migration**

In [ ]:
#Cargamos nuestro dataset
df_habilidades = wr.s3.read_csv(path='s3://data-fm-05/raw_data/skill_migration_public.csv', boto3_session=my_session)

#Hacemos transformaciones
df_habilidades.drop(['wb_income', 'wb_region', 'skill_group_id'], axis=1, inplace=True )
df_habilidades.drop(columns=df_habilidades.columns[9:26],axis=1,inplace= True)
df_habilidades.rename(columns={'country_code ':'codigo_pais','country_name ':'pais',
                          'skill_group_category':'categoria', 'skill_group_name':'habilidad',
                          'net_per_10K_2015': '2015', 'net_per_10K_2016': '2016', 'net_per_10K_2017': '2017',
                          'net_per_10K_2018': '2018', 'net_per_10K_2019': '2019'}, inplace=True)
df_habilidades= df_habilidades.drop_duplicates()

#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=df_habilidades, path='s3://data-fm-05/clean_data/skill_migration_public.csv', boto3_session=my_session)

**Tabla de empleo, desempleo y participacion**

In [ ]:
#Cargamos nuestro dataset
df_Porcentaje_empleo_desempleo_participacion = wr.s3.read_csv(path="s3://data-fm-05/raw_data/Employment_unemployment_and_participation.csv", boto3_session=my_session)

#Hacemos transformaciones
df_Porcentaje_empleo_desempleo_participacion.drop(['BIRTH', 'GENDER', 'RATE', 'YEAR', 'Flag Codes', 'Flags'], axis=1, inplace= True)
df_Porcentaje_empleo_desempleo_participacion.drop(columns=df_Porcentaje_empleo_desempleo_participacion.columns[6:12],axis=1,inplace= True)
df_Porcentaje_empleo_desempleo_participacion.rename(columns={'COUNTRY':'codigo_pais','Country ':'pais',
                          'Place of birth':'lugar_de_nacimiento', 'Gender':'genero',
                          'Rate': 'referencia_tasas', 'Year': 'anio', 'Value': 'valor'}, inplace=True)
df_Porcentaje_empleo_desempleo_participacion= df_Porcentaje_empleo_desempleo_participacion[(df_Porcentaje_empleo_desempleo_participacion['genero']== 'Total')]
df_Porcentaje_empleo_desempleo_participacion.drop(['genero'], axis=1, inplace= True)

#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=df_Porcentaje_empleo_desempleo_participacion, path='s3://data-fm-05/clean_data/Employment_unemployment_and_participation.csv', boto3_session=my_session)

**Tabla de Tasa de participacion**

In [ ]:
#Cargamos nuestro dataset
df_participacion = wr.s3.read_csv(path='s3://data-fm-05/raw_data/Tasa_participacion_fuerza_trabajo.csv', boto3_session=my_session)

#Hacemos transformaciones
df_participacion.drop(['source', 'source.label', 'indicator', 'indicator.label','sex', 'sex.label', 'classif1', 'classif2'], axis=1, inplace= True)
df_participacion.drop(columns=df_participacion.columns[6:15],axis=1,inplace= True)
df_participacion.rename(columns={'ref_area':'codigo_pais','ref_area.label':'pais',
                          'classif1.label':'edad', 'classif2.label':'lugar_de_nacimiento',
                          'time': 'anio', 'obs_value': 'valor'}, inplace=True)
df_participacion= df_participacion[(df_participacion['edad']== 'Edad (Tramos agregados): Total')]
df_participacion.drop(['edad'], axis=1, inplace=True)

#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=df_participacion, path='s3://data-fm-05/clean_data/Tasa_participacion_fuerza_trabajo.csv', boto3_session=my_session)

**Tabla de migracion neta banco mundial**

In [ ]:
#Cargamos nuestro dataset
df_migracion_neta = wr.s3.read_csv(path='s3://data-fm-05/raw_data/migracion_neta.csv', boto3_session=my_session)

#Hacemos transformaciones
df_migracion_neta.drop(['Indicator Name', 'Indicator Code', 'Unnamed: 66'], axis=1, inplace= True)
df_migracion_neta.drop(columns=df_migracion_neta.columns[2:32],axis=1,inplace= True)
df_migracion_neta.rename(columns={'Country Name':'pais','Country Code':'codigo_pais'}, inplace=True)
df_migracion_neta.drop([110], axis=0, inplace=True)

#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=df_migracion_neta, path='s3://data-fm-05/clean_data/migracion_neta.csv', boto3_session=my_session)

**Fact Empleo Condicion**

In [ ]:
#Cargamos nuestro dataset
Porcentaje_empleo_desempleo_participacion = wr.s3.read_csv(path="s3://data-fm-05/raw_data/Employment_unemployment_and_participation.csv", boto3_session=my_session)

#Hacemos transformaciones
Porcentaje_empleo_desempleo_participacion.drop(['BIRTH', 'GENDER', 'RATE', 'YEAR', 'Flag Codes', 'Flags'], axis=1, inplace= True)
Porcentaje_empleo_desempleo_participacion.drop(columns=Porcentaje_empleo_desempleo_participacion.columns[6:12],axis=1,inplace= True)
Porcentaje_empleo_desempleo_participacion.rename(columns={'COUNTRY':'codigo_pais','Country':'pais',
                          'Place of birth':'lugar_de_nacimiento', 'Gender':'genero',
                          'Rate': 'referencia_tasas', 'Year': 'anio', 'Value': 'valor'}, inplace=True)
Porcentaje_empleo_desempleo_participacion= Porcentaje_empleo_desempleo_participacion[(Porcentaje_empleo_desempleo_participacion['genero']== 'Total')]
Porcentaje_empleo_desempleo_participacion.drop(['genero'], axis=1, inplace= True)
Porcentaje_empleo_desempleo_participacion_empleados = Porcentaje_empleo_desempleo_participacion[Porcentaje_empleo_desempleo_participacion['referencia_tasas'] == 'Employment rate'].rename(columns={'valor':"empleados"}).drop(['referencia_tasas'], axis=1)
Porcentaje_empleo_desempleo_participacion_desempleados = Porcentaje_empleo_desempleo_participacion[Porcentaje_empleo_desempleo_participacion['referencia_tasas'] == 'Unemployment rate'].rename(columns={'valor':"desempleados"}).drop(['referencia_tasas'], axis=1)
Porcentaje_empleo_desempleo_participacion_participacion = Porcentaje_empleo_desempleo_participacion[Porcentaje_empleo_desempleo_participacion['referencia_tasas'] == 'Participation rate'].rename(columns={'valor':"participacion"}).drop(['referencia_tasas'], axis=1)


merge1 = pd.merge(Porcentaje_empleo_desempleo_participacion_empleados, Porcentaje_empleo_desempleo_participacion_desempleados, on=['codigo_pais','pais','anio','lugar_de_nacimiento'], how='outer')
merge2 = pd.merge(merge1, Porcentaje_empleo_desempleo_participacion_participacion, on=['codigo_pais','pais','anio','lugar_de_nacimiento'], how='outer')
merge2['lugar_de_nacimiento'].replace(to_replace='Foreign-born', value='extranjero', inplace=True)
merge2['lugar_de_nacimiento'].replace(to_replace='Native-born', value='nativo', inplace=True)

participacion = wr.s3.read_csv(path='s3://data-fm-05/raw_data/Tasa_participacion_fuerza_trabajo.csv', boto3_session=my_session)
participacion.drop(['source', 'source.label', 'indicator', 'indicator.label','sex', 'sex.label', 'classif1', 'classif2'], axis=1, inplace= True)
participacion.drop(columns=participacion.columns[6:15],axis=1,inplace= True)
participacion.rename(columns={'ref_area':'codigo_pais','ref_area.label':'pais',
                          'classif1.label':'edad', 'classif2.label':'lugar_de_nacimiento',
                          'time': 'anio', 'obs_value': 'tasa_participacion'}, inplace=True)
participacion= participacion[(participacion['edad']== 'Edad (Tramos agregados): Total')]
participacion.drop(['edad'], axis=1, inplace=True)
participacion['lugar_de_nacimiento'].replace(to_replace='Lugar de nacimiento: Nativo-nacido', value='nativo', inplace=True)
participacion['lugar_de_nacimiento'].replace(to_replace='Lugar de nacimiento: Nacido en el extranjero', value='extranjero', inplace=True)

fact_empleo_condicion = pd.merge(merge2, participacion, on=['codigo_pais','pais','lugar_de_nacimiento','anio'], how='outer')


#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=fact_empleo_condicion, path="s3://data-fm-05/clean_data/fact_empleo_condicion.csv", boto3_session=my_session)

**Fact Ind Econonómico**

In [45]:
pib = df_PIB.melt(id_vars=['pais','codigo_pais'], var_name='anio', value_name='PIB')
pib['anio'] = pib['anio'].astype('int64')

#Cargamos nuestro dataset
df_salario_minimo = wr.s3.read_csv(path='s3://data-fm-05/raw_data/salario_minimo_mensual_bruto_anual_dolares.csv', sep=',', header=0, boto3_session=my_session)

#Hacemos transformaciones
df_salario_minimo.drop('source', axis=1, inplace=True)
df_salario_minimo.drop('source.label', axis=1, inplace=True)
df_salario_minimo.drop('indicator', axis=1, inplace=True)
df_salario_minimo.drop('indicator.label', axis=1, inplace=True)
df_salario_minimo.drop('classif1', axis=1, inplace=True)
df_salario_minimo.drop('classif1.label', axis=1, inplace=True)
df_salario_minimo.drop('obs_status', axis=1, inplace=True)
df_salario_minimo.drop('obs_status.label', axis=1, inplace=True)
df_salario_minimo.drop('note_indicator', axis=1, inplace=True)
df_salario_minimo.drop('note_indicator.label', axis=1, inplace=True)
df_salario_minimo.drop('note_source', axis=1, inplace=True)
df_salario_minimo.drop('note_source.label', axis=1, inplace=True)
df_salario_minimo.rename(columns={'ref_area':'codigo_pais', 'ref_area.label':'pais', 'time':'anio', 'obs_value':'salario_minimo'}, inplace=True)
df_salario_minimo = df_salario_minimo[(df_salario_minimo['anio'] >= 1990)]

#Cargamos nuestro dataset
df_average_wages = wr.s3.read_csv(path='s3://data-fm-05/raw_data/average_wages.csv',sep=',', header=0, boto3_session=my_session)

#Hacemos transformaciones
df_average_wages.drop('INDICATOR', axis=1, inplace=True)
df_average_wages.drop('SUBJECT', axis=1, inplace=True)
df_average_wages.drop('MEASURE', axis=1, inplace=True)
df_average_wages.drop('FREQUENCY', axis=1, inplace=True)
df_average_wages.drop('Flag Codes', axis=1, inplace=True)
df_average_wages.rename(columns={'LOCATION':'codigo_pais', 'TIME':'anio', 'Value':'salario_promedio'}, inplace=True)

#Cargamos nuestro dataset
df_wage_levels = wr.s3.read_csv(path='s3://data-fm-05/raw_data/wage_levels.csv', sep=',', header=0, boto3_session=my_session)

#Hacemos transformaciones
df_wage_levels.drop('INDICATOR', axis=1, inplace=True)
df_wage_levels.drop('MEASURE', axis=1, inplace=True)
df_wage_levels.drop('FREQUENCY', axis=1, inplace=True)
df_wage_levels.drop('Flag Codes', axis=1, inplace=True)
df_wage_levels = df_wage_levels[(df_wage_levels['TIME'] >= 1990)]
df_wage_levels_low = df_wage_levels[df_wage_levels['SUBJECT'] == 'LPAY'].rename(columns={'Value':'ingresos_altos'}).drop(['SUBJECT'], axis=1)
df_wage_levels_high = df_wage_levels[df_wage_levels['SUBJECT'] == 'HPAY'].rename(columns={'Value':'ingresos_bajos'}).drop(['SUBJECT'], axis=1)

df_wage_levels_merged = pd.merge(df_wage_levels_high, df_wage_levels_low, on=['LOCATION','TIME'], how='outer')
df_wage_levels_merged.rename(columns={'LOCATION':'codigo_pais', 'SUBJECT':'tipo', 'TIME': 'anio'}, inplace=True)

#Cargamos nuestro dataset
agricultura = wr.s3.read_csv(path='s3://data-fm-05/raw_data/empleos_agricultura.csv', boto3_session=my_session)

#Hacemos transformaciones
agricultura.drop(['Indicator Name','Indicator Code',"1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989",'1990','2020','2021','Unnamed: 66'], axis=1, inplace= True)
agricultura = agricultura.melt(id_vars=['Country Name','Country Code'], var_name='anio', value_name='%empleo_agricultura')
agricultura.rename(columns={'Country Name':'pais','Country Code':'codigo_pais'}, inplace=True)
agricultura['anio'] = agricultura['anio'].astype('int64')

#Cargamos nuestro dataset
industria = wr.s3.read_csv(path= 's3://data-fm-05/raw_data/empleos_industria.csv', boto3_session=my_session)

#Hacemos transformaciones
industria.drop(['Indicator Name','Indicator Code',"1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989",'1990','2020','2021','Unnamed: 66'], axis=1, inplace= True)
industria = industria.melt(id_vars=['Country Name','Country Code'], var_name='anio', value_name='%empleo_industria')
industria.rename(columns={'Country Name':'pais','Country Code':'codigo_pais'}, inplace=True)
industria['anio'] = industria['anio'].astype('int64')

#Cargamos nuestro dataset
servicio = wr.s3.read_csv(path='s3://data-fm-05/raw_data/empleos_servicio.csv', boto3_session=my_session)

#Hacemos transformaciones
servicio.drop(['Indicator Name','Indicator Code',"1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989",'1990','2020','2021','Unnamed: 66'], axis=1, inplace= True)
servicio = servicio.melt(id_vars=['Country Name','Country Code'], var_name='anio', value_name='%empleo_servicio')
servicio.rename(columns={'Country Name':'pais','Country Code':'codigo_pais'}, inplace=True)
servicio['anio'] = servicio['anio'].astype('int64')

#Realizamos un merge de varios dataframes para crear uno completo
merge1 = pd.merge(pib, df_salario_minimo, on=['codigo_pais','pais','anio'], how='outer')
merge2 = pd.merge(merge1, df_average_wages, on=['codigo_pais','anio'], how='outer')
merge3 = pd.merge(merge2, df_wage_levels_merged, on=['codigo_pais','anio'], how='outer')
merge4 =pd.merge(merge3, agricultura, on=['codigo_pais','pais','anio'], how='outer')
merge5 = pd.merge(merge4, industria, on=['codigo_pais','pais','anio'], how='outer')

fact_ind_economico = pd.merge(merge5, servicio, on=['codigo_pais','pais','anio'], how='outer')


#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=fact_ind_economico, path="s3://data-fm-05/clean_data/fact_ind_economico.csv", boto3_session=my_session)

**Fact Ind Econonómico**

In [49]:
#Cargamos nuestro dataset
df_derechos_laborales = wr.s3.read_csv(path="s3://data-fm-05/raw_data/regulacion_juridica_al_empleo.csv", sep=',', header=0, boto3_session=my_session)

#Hacemos transformaciones
df_derechos_laborales.drop('source', axis=1, inplace=True)
df_derechos_laborales.drop('source.label', axis=1, inplace=True)
df_derechos_laborales.drop('indicator', axis=1, inplace=True)
df_derechos_laborales.drop('indicator.label',axis=1, inplace=True)
df_derechos_laborales.rename(columns={'ref_area':'codigo_pais', 'ref_area.label':'pais','time':'anio','obs_value':'derechos_laborales'}, inplace=True)

#Cargamos nuestro dataset
df_human_rights = wr.s3.read_csv(path='s3://data-fm-05/raw_data/distribution-human-rights.csv', sep=',', header=0, boto3_session=my_session)

#Hacemos transformaciones
df_human_rights.rename(columns={'Entity':'pais', 'Code':'codigo_pais', 'Year':'anio', 'civ_libs_vdem_owid': 'derechos_humanos'}, inplace=True)
df_human_rights.dropna(subset=['derechos_humanos'], inplace=True)
df_human_rights.drop('codigo_pais', axis=1, inplace=True)
df_human_rights = df_human_rights[(df_human_rights['anio'] >= 1990)]

#Cargamos nuestro dataset
df_gender_wage_gap = wr.s3.read_csv(path='s3://data-fm-05/raw_data/gender_wage_gap.csv', sep=',', header=0, boto3_session=my_session)

#Hacemos transformaciones
df_gender_wage_gap.drop('INDICATOR', axis=1, inplace=True)
df_gender_wage_gap.drop('MEASURE', axis=1, inplace=True)
df_gender_wage_gap.drop('FREQUENCY', axis=1, inplace=True)
df_gender_wage_gap.drop('Flag Codes', axis=1, inplace=True)
df_gender_wage_gap.rename(columns={'LOCATION':'codigo_pais', 'SUBJECT':'tipo', 'TIME':'anio', 'Value':'valor'}, inplace=True)
df_gender_wage_gap = df_gender_wage_gap[(df_gender_wage_gap['anio'] >= 1990)]
df_gender_wage_gap_employee = df_gender_wage_gap[df_gender_wage_gap['tipo'] == 'EMPLOYEE'].rename(columns={'valor':'brecha_salarial_empleo'}).drop(['tipo'], axis=1)
df_gender_wage_gap_selfemployed = df_gender_wage_gap[df_gender_wage_gap['tipo'] == 'SELFEMPLOYED'].rename(columns={'valor':'brecha_salarial_autoempleo'}).drop(['tipo'], axis=1)
df_gender_wage_gap_merged = pd.merge(df_gender_wage_gap_employee, df_gender_wage_gap_selfemployed, on=['codigo_pais','anio'], how='outer')

#Cargamos nuestro dataset
df_gini = wr.s3.read_csv(path='s3://data-fm-05/raw_data/gini.csv', boto3_session=my_session)

#Hacemos transformaciones
df_gini.rename(columns={'Code':'codigo_pais', 'Entity':'pais', 'Year':'anio', 'Gini coefficient':'coeficiente_gini'}, inplace=True)

#Realizamos un merge de varios dataframes para crear uno completo
merge1 = pd.merge(df_derechos_laborales, df_human_rights, on=['pais','anio'], how='outer')
merge2 = pd.merge(merge1, df_gini, on=['codigo_pais','pais','anio'], how='outer')

fact_ind_sociopolitico = pd.merge(merge2, df_gender_wage_gap_merged, on=['codigo_pais',	'anio'], how='outer')


#Enviamos el dataset limpio a otro bucket
wr.s3.to_csv(df=fact_ind_sociopolitico, path="s3://data-fm-05/clean_data/fact_ind_sociopolitico.csv", boto3_session=my_session)

#### Carga de Datos a Redshift

In [ ]:
#Establecemos conexión con la base de datos
con = wr.redshift.connect(connection="connection", boto3_session=my_session)

In [ ]:
#Leemos el dataset limpio
fact_empleo_condicion = wr.s3.read_csv(path="s3://data-fm-05/clean_data/fact_empleo_condicion.csv", boto3_session=my_session)

#Cargamos la tabla a la base de datos
wr.redshift.copy(
    df = fact_empleo_condicion,
    path = "s3://path",
    con=con,
    table="fact_empleo_condicion",
    schema= "public",
    iam_role= "iam_role",
    boto3_session=my_session,
    mode= "overwrite"
)

In [ ]:
#Leemos el dataset limpio
fact_ind_economico = wr.s3.read_csv(path="s3://data-fm-05/clean_data/fact_ind_economico.csv", boto3_session=my_session)

#Cargamos la tabla a la base de datos
wr.redshift.copy(
    df = fact_ind_economico,
    path = "s3://path",
    con=con,
    table="fact_ind_economico",
    schema= "public",
    iam_role= "iam_role",
    boto3_session=my_session
)

In [ ]:
#Leemos el dataset limpio
fact_ind_sociopolitico = wr.s3.read_csv(path="s3://data-fm-05/clean_data/fact_ind_sociopolitico.csv", boto3_session=my_session)

#Cargamos la tabla a la base de datos
wr.redshift.copy(
    df = fact_ind_sociopolitico,
    path = "s3://path",
    con=con,
    table="fact_ind_sociopolitico",
    schema= "public",
    iam_role= "iam_role",
    boto3_session=my_session
)

In [ ]:
#Leemos el dataset limpio
fact_migracion_industria = wr.s3.read_csv(path="s3://data-fm-05/clean_data/industry_migration_public.csv", boto3_session=my_session)

#Cargamos la tabla a la base de datos
wr.redshift.copy(
    df = fact_migracion_industria,
    path = "s3://path",
    con=con,
    table="fact_ind_sociopolitico",
    schema= "public",
    iam_role= "iam_role",
    boto3_session=my_session
)

In [ ]:
#Leemos el dataset limpio
fact_migracion_skills = wr.s3.read_csv(path="s3://data-fm-05/clean_data/skill_migration_public.csv", boto3_session=my_session)

#Cargamos la tabla a la base de datos
wr.redshift.copy(
    df = fact_migracion_skills,
    path = "s3://path",
    con=con,
    table="fact_ind_sociopolitico",
    schema= "public",
    iam_role= "iam_role",
    boto3_session=my_session
)

In [ ]:
#Leemos el dataset limpio
dim_pais = wr.s3.read_csv(path="s3://data-fm-05/clean_data/dim_pais.csv", boto3_session=my_session)

#Cargamos la tabla a la base de datos
wr.redshift.copy(
    df = dim_pais,
    path = "s3://path",
    con=con,
    table="fact_ind_sociopolitico",
    schema= "public",
    iam_role= "iam_role",
    boto3_session=my_session
)

In [ ]:
dim_calendario = wr.s3.read_csv(path="s3://data-fm-05/clean_data/dim_calendario.csv", boto3_session=my_session)

#Cargamos la tabla a la base de datos
wr.redshift.copy(
    df = dim_calendario,
    path = "s3://path",
    con=con,
    table="fact_ind_sociopolitico",
    schema= "public",
    iam_role= "iam_role",
    boto3_session=my_session
)

Finalizamos la conexión

In [ ]:
con.close()